In [1]:
# from pygasflow.utils.decorators import check
# from pygasflow.utils.roots import apply_bisection
from scipy.optimize import bisect
import numpy as np



In [2]:
# TODO:
# 1. In the functions where the bisection is used, evaluate if the initial mach range
#   for supersonic case is sufficient for all gamma and ratios.
# 2. Evaluate if scipy.brentq and scipy.brenth performs better than scipy.bisect
# 3. Provide tolerance and maxiter arguments to the function where bisect is used.

# NOTE: certain function could very well be computed by using other functions.
#   For instance, Critical_Temperature_Ratio could be computed using Temperature_Ratio.
#   In doing so, a performance penalty is introduced, that could become significant when
#   computing millions of data points. This is mostly the case with functions that get
#   called by root finding methods.
#   Therefore, I use plain formulas as much as possible.

class isentropic:
    # @check
    def critical_velocity_ratio( M, gamma=1.4):
        """
        Compute the critical velocity ratio U/U*.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Critical velocity ratio U/U*.
        """
        # need to deal with division by 0
        ratio = np.zeros_like(M)
        idx = M != 0
        ratio[idx] = 1 / ((2 / (gamma + 1))**(1 / (gamma - 1)) / M[idx]
                          * (1 + (gamma - 1) / 2 * M[idx]**2)**0.5)
        return ratio


    # @check
    def critical_temperature_ratio( M, gamma=1.4):
        """
        Compute the critical temperature ratio T/T*.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Critical Temperature ratio T/T*.
        """
        # return Temperature_Ratio.__no_check(M, gamma) * 0.5 * (gamma + 1)
        return ((gamma + 1) / 2) / (1 + (gamma - 1) / 2 * M**2)


    # @check
    def critical_pressure_ratio( M, gamma=1.4):
        """
        Compute the critical pressure ratio P/P*.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Critical Pressure ratio P/P*.
        """
        # return Pressure_Ratio.__no_check(M, gamma) * (0.5 * (gamma + 1))**(gamma / (gamma - 1))
        return (((gamma + 1) / 2) / (1 + (gamma - 1) / 2 * M**2))**(gamma / (gamma - 1))



    # @check
    def critical_area_ratio( M, gamma=1.4):
        """
        Compute the critical area ratio A/A*.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Critical area ratio A/A*.
        """
        # return 1  / Critical_Density_Ratio.__no_check(M, gamma) * np.sqrt(1 / Critical_Temperature_Ratio.__no_check(M, gamma)) / M
        # TODO: here, division by M=0 produce the correct results, infinity.
        # Do I need to suppress the warning???
        return (((1 + (gamma - 1) / 2 * M**2) / ((gamma + 1) / 2))**((gamma + 1) / (2 * (gamma - 1)))) / M


    # @check
    def pressure_ratio( M, gamma=1.4):
        """
        Compute the pressure ratio P/P0.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Pressure ratio P/P0.
        """
        return (1 + (gamma - 1) / 2 * M**2)**(-gamma / (gamma - 1))


    # @check
    def temperature_ratio( M, gamma=1.4):
        """
        Compute the temperature ratio T/T0.
        Parameters
        ----------
            M : array_like
                Mach number. If float, list, tuple is given as input, a conversion
                will be attempted. Must be M > 0.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Temperature ratio T/T0.
        """
        return 1 / (1 + (gamma - 1) / 2 * M**2)




    # @check
    def m_from_pressure_ratio( ratio, gamma=1.4):
        """
        Compute the Mach number given the Isentropic Pressure Ratio P/P0.
        Parameters
        ----------
            ratio : array_like
                Isentropic Pressure Ratio P/P0. If float, list, tuple is given as input,
                a conversion will be attempted. Must be 0 <= P/P0 <= 1.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Mach Number.
        """
        if np.any(ratio < 0) or np.any(ratio > 1):
            raise ValueError("Pressure ratio must be 0 <= P/P0 <= 1.")
        return np.sqrt(2 / (gamma - 1) * (1 / ratio**((gamma - 1) / gamma) - 1))




    # @check
    def m_from_critical_area_ratio( ratio, flag="sub", gamma=1.4):
        """
        Compute the Mach number given the Isentropic Critical Area Ratio A/A*.
        Parameters
        ----------
            ratio : array_like
                Isentropic Critical Area Ratio A/A*. If float, list, tuple is given as input,
                a conversion will be attempted. Must be A/A* >= 1.
            flag : string
                Can be either 'sub' (subsonic) or 'sup' (supersonic). Default to 'sub'.
            gamma : float
                Specific heats ratio. Default to 1.4. Must be > 1.

        Returns
        -------
            out : ndarray
                Mach Number.
        """
        if np.any(ratio < 1):
            raise ValueError("Area ratio must be A/A* >= 1.")

        def func(M, r): return r - (((1 + (gamma - 1) / 2 * M**2) /
                                     ((gamma + 1) / 2))**((gamma + 1) / (2 * (gamma - 1)))) / M
        # func = lambda M, r: r - Critical_Area_Ratio.__no_check(M, gamma)
        return isentropic.apply_bisection(ratio, func, flag=flag)

    # with arguments True, I want to convert to np.ndarray the first two parameters


    def apply_bisection( ratio, func, flag="sub"):
        """ Helper function used for applying the bisection method to find the
        roots of a given function.
        Args:
            ratio:  Ratio (or parameter) passed to the function.
            func:   Function returning a number.
            flag:   Can be either "sub" or "super".

        Return:
            The zero of the given function.
        """
        if flag == "sub":
            mach_range = [np.spacing(1), 1]
        else:
            # realmax = np.finfo(np.float64).max

            # TODO: evaluate if this mach range is sufficient for all gamma and ratios.
            mach_range = [1, 100]

        M = np.zeros_like(ratio)
        if hasattr(ratio, '__iter__'):

            for i, r in enumerate(ratio):
                M[i] = bisect(func, *mach_range, args=(r))
            return M
        M = bisect(func, *mach_range, args=(ratio))
        return M

In [3]:
from scipy.integrate import RK45
import numpy as np
from numpy import sqrt, pi, cos, power
import pandas as pd
from math import log, ceil
# import isentropic
# import chart_studio.plotly as py
# import plotly.graph_objs as go
# import plotly
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

cf.set_config_file(offline=True)
cf.set_config_file(theme='white', sharing='public', offline=False)

In [4]:
class Case:
    def __init__(self, diam, leng):
        self.diam = diam
        self.leng = leng
        self.volu = pi/4*diam**2*leng


class Grain:
    def __init__(self, case, core0, leng0):
#         self.case = case
        self.diam = case.diam
        self.core0 = core0
        self.leng0 = leng0
        self.web0 = (self.diam-self.core0)/2
        self.cross_sec_area = pi/4*self.diam**2
        
        # set initial parameters
        self.calc_params(0)
        
    def calc_params( self, x ):
        self.calc_core(x)
        self.calc_core_area(x)
        self.calc_anulus_area(x)
        self.calc_length(x)
        self.calc_area(x)
        self.calc_volume(x)

            
    def calc_core(self, x):
        self.core = self.core0+2*x
    
    def calc_core_area(self, x):
        self.core_area = pi/4 * self.core**2
    
    def calc_anulus_area( self, x):
        self.anulus_area = self.cross_sec_area - self.core_area
    
    def calc_length(self, x):
        self.length = self.leng0-2*x
        
    def calc_area(self, x):
        self.area = pi * self.core * self.length + 2*self.anulus_area
    
    def calc_volume(self, x):
        self.volume = pi/4 * (self.diam**2 - self.core**2) * self.length
        if self.volume <= 0:
            self.core = self.diam
            self.length = self.leng0 - 2*self.diam
            self.anulus_area = 0
            self.core_area = self.cross_sec_area
            self.area = 0
            self.volume = 0
                    
class Grains:
    def __init__(self, grains=[]):
        if len(grains) == 0:
            return ValueError("must have at least 1 propellant grain")
        self.grains = grains
        self.num_grains = len(grains)
    
    def calc_params( self, x ):
        for grain in self.grains:
            grain.calc_params(x)
        self.calc_area()
        self.calc_volume()
        self.calc_area_grains_end()
        self.calc_kn_grains_end()
        
    
    def calc_area( self ):
        self.area = sum([grain.area for grain in self.grains])
        
    def get_burning_area(self):
        return self.area
    
    def calc_volume( self ):
        self.volume = sum([grain.volume for grain in self.grains])
        if self.volume <= 0:
            self.volume = 0
            self.area = 0
    
    def get_volume(self):
        return self.volume
    
    def web_regress_check( self, dx_dt ):
        """
        Checks if there are any grians still burning, if so allows dx_dt to continue, else dx_dt = 0
        Parameters:
        dx_dt float, web regress rate
        returns dx_dt
        """
        if self.volume <= 0: return 0
        return dx_dt
    
    def calc_area_grains_end( self ):
        """
        calculates the area exposed above each grains bottom surface
        """
        area_grains_above = 0
        area_grains_end = []
        for grain in grains:
            # area of grain minus the end
            area_grain_end = grain.core_area * grain.length + grain.anulus_area
            # add that area with the full area of the grians above
            area_grains_end.append( area_grain_end + area_grains_above )
            area_grains_above += grain.area
        self.area_grains_end = area_grains_end
    
    def get_area_grains_end(self):
        return self.area_grains_end
    
    def calc_kn_grains_end( self ):
        self.kn_grains_end =  [\
                area_grain_end / area_core for area_grain_end, area_core \
                in zip( self.area_grains_end, [grain.core for grain in self.grains] )\
               ]
    def get_kn_grains_end( self ):
        return self.kn_grains_end

class Bates(Grain):
    def __init__(self, case, core0):
        self.case = case
        self.diam = case.diam
        self.core0 = core0
#         self.leng0 = (3*self.diam - self.core0) / 2
#         print(self.leng0)
        self.web0 = (self.diam-self.core0)/2


class Propellant:
    press_at = 101325

    def __init__(self, brc=.0001, bre=.3, dens=1500, k=1.4, r=287, temp=2000):

        self.burn_rate_coeff = brc
        self.bun_rate_exp = bre
        self.dens = dens
        self.k = k
        self.r = r
        self.temp = temp
        self.press_crit = ((k-1)/2+1)**(k/(k-1))

    def burnrate(self, press):
        if press <= 0:
            return 0
        return self.burn_rate_coeff*power(press, self.bun_rate_exp)

    def mach(self, press):

        if press <= 0:
            return 0
        press_ratio = self.press_at/press
#         print(press_ratio)
        if press_ratio >= 1:
            return 0
        if press_ratio <= 0:
            return 1
        mach_ideal = isentropic.m_from_pressure_ratio(press_ratio, gamma=self.k)
        return min(1, mach_ideal)

    def isentropic(self, press, expand):
        # temp_exit_ratio is used to calculate exit velocity
        # mach at exit ideally expanded Valid for sub and super

        if press <= self.press_at:
            return 0, 1, 1
        
        mach_ideal = isentropic.m_from_pressure_ratio(self.press_at/press, gamma=self.k)
        expand_ideal = isentropic.critical_area_ratio(mach_ideal, gamma=self.k)

        if mach_ideal < 1:  # if subsonic
            if expand > expand_ideal:  # over expanded
                mach_exit = mach_ideal
                press_exit_rato = isentropic.pressure_ratio(
                    mach_exit, gamma=self.k) * expand_ideal/expand
                temp_exit_ratio = isentropic.temperature_ratio(mach_exit, gamma=self.k)
                return mach_exit, press_exit_rato, temp_exit_ratio

            if expand <= expand_ideal:  # under expanded (the better one)
                mach_exit = isentropic.m_from_critical_area_ratio(expand, flag='sub', gamma=self.k)
                press_exit_rato = isentropic.pressure_ratio(mach_exit, gamma=self.k)
                temp_exit_ratio = isentropic.temperature_ratio(mach_exit, gamma=self.k)
                return mach_exit, press_exit_rato, temp_exit_ratio

        if expand > expand_ideal:  # over expanded
            mach_exit = mach_ideal
            press_exit_rato = isentropic.pressure_ratio(
                mach_exit, gamma=self.k) * expand_ideal/expand
            temp_exit_ratio = isentropic.temperature_ratio(mach_exit, gamma=self.k)
            return mach_exit, press_exit_rato, temp_exit_ratio

        if expand <= expand_ideal:  # under expanded (the better one)
            mach_exit = isentropic.m_from_critical_area_ratio(expand, flag='sup', gamma=self.k)
            press_exit_rato = isentropic.pressure_ratio(mach_exit, gamma=self.k)
            temp_exit_ratio = isentropic.temperature_ratio(mach_exit, gamma=self.k)
            return mach_exit, press_exit_rato, temp_exit_ratio

    def mass_flux(self, press):

        k = self.k
        km1 = k-1
        kp1 = k+1
        mach = self.mach(press)
        mass_flux = press / sqrt(self.temp) * sqrt(k/self.r) \
            * mach * (1 + km1/2 * mach**2) ** (-kp1/(2*km1))

        if mass_flux < 0:  # mass flux should always be positive
            return 0, 0
        return mach, mass_flux

    def dpress_dt(self, dmass_dt, dvol_dt, vol, press):
        return (self.r * self.temp * dmass_dt - dvol_dt * press) / vol

    def speed_of_sound(self, temp_ratio):
        return sqrt(self.k * self.r * temp_ratio * self.temp)


class Nozzle:
    def __init__(self, diam, expand, div, efficiency):
        self.diam = diam
        self.expand = expand
        self.div_angle = div

        self.area = pi/4*diam**2
        self.area_exit = self.area*expand
        
        self.efficiency = efficiency
        self.cos_losses = (1 + cos(self.div_angle))/2
        
    def optimize_expand( self, prop, press ):
        mach_ideal = isentropic.m_from_pressure_ratio(prop.press_at/press, gamma=prop.k)
        expand_ideal = isentropic.critical_area_ratio(mach_ideal, gamma=prop.k)
        self.expand = expand_ideal
        return self.expand


class Motor:
    press_at = 101325  # pa
    state_data_headers =   [
                    't', 'x', 'press', 'mass_burn', 'mass_exit', 'vol_grains', 'vol_empty',
                    'mach', 'mass_flux', 'mach_exit', 'exit_velocity', 'thrust', 'thrust_press', 'thrust_flow', 'press_exit'
                   ]

    dstate_data_headers =  [
                    't', 'dx_dt', 'dpress_dt', 'dmass_burn_dt', 'dmass_exit_dt',
                    'dvol_grains_dx', 'dvol_empty_dx', 'dvol_grains_dt', 'dvol_empty_dt',
                   ]

    tspan = 100
    press_init = 2  # atm's must start motor at above atmospheric pressure

    def __init__(self, case, grains, prop, nozzle, mod=1):
        self.case = case
        self.prop = prop
        self.nozzle = nozzle
        
        self.grains = Grains(grains)
        self.grains.calc_params(0)

        vol_empty = self.case.volu - self.grains.get_volume()
        if vol_empty <= 0:
            raise ValueError("Case Empty Volume cannot be less than 0. Check Case length parameter")
        if self.press_init <= 1:
            raise ValueError("Initial Pressure must be greater than atmospheric")
        
        self.mod = mod
    
    def get_prop_mass( self, x=0):
        self.grains.calc_params(x)
        prop_mass = self.prop.dens * self.grains.get_volume()
        return prop_mass

    def physics(self, t, stateVars):
        x = self.mod*stateVars[0]
        press = stateVars[1]
        mass_burn = stateVars[2]
        mass_exit = stateVars[3]
        
        mass_gas = mass_burn - mass_exit

        # burn rate / web regres rate calculation
        dx_dt = self.prop.burnrate(press)
        
        self.grains.calc_params(x)
        dx_dt = self.grains.web_regress_check(dx_dt)

        # volumetric calculations
        vol_grains = self.grains.get_volume()
        vol_empty = self.case.volu - vol_grains
        dvol_grains_dx = -self.grains.get_burning_area()
        dvol_empty_dx = -dvol_grains_dx
        dvol_grains_dt = dx_dt * dvol_grains_dx*self.mod
        dvol_empty_dt = dx_dt * dvol_empty_dx
        
#         area_grains_end = self.grains.get_area_grains_end()
        mass_flux_grain_end_data = [\
                                kn_grain_end*self.prop.dens*dx_dt\
                                for kn_grain_end in self.grains.get_kn_grains_end()\
                               ]
        
        # delta mass burnt
        dmass_burn_dt = - self.prop.dens * dvol_grains_dt

        # delta mass exit is a function of pressure
        mach, mass_flux = self.prop.mass_flux(press)
        dmass_exit_dt = self.nozzle.area * mass_flux

        # change in pressure calculated from dirivitive form of ideal gas law
        dpress_dt = self.prop.dpress_dt(dmass_burn_dt - dmass_exit_dt,
                                        dvol_empty_dt, vol_empty, press)

        # gas exit conditions and thrust calc
        mach_exit, press_exit_rato, temp_exit_ratio = self.prop.isentropic(
            press, self.nozzle.expand)
        exit_velocity = mach_exit * self.prop.speed_of_sound(temp_exit_ratio)
        
        press_exit = press_exit_rato * press 
        thrust_press = (press_exit - self.press_at) * self.nozzle.area_exit
        thrust_flow = dmass_exit_dt * exit_velocity * self.nozzle.cos_losses
        
        thrust = self.nozzle.efficiency * (thrust_flow + thrust_press )
        
        state_data = [
            t, x, press, mass_burn, mass_exit, vol_grains, vol_empty,
            mach, mass_flux, mach_exit, exit_velocity, thrust, thrust_press, thrust_flow, press_exit
        ]

        dstate_data = [
            t, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
            dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
            ]
        
        mass_flux_grain_end_data = [t] + mass_flux_grain_end_data
        return state_data, dstate_data, mass_flux_grain_end_data

    def dydt(self, t, stateVars):
        _, dstate_data, _ = self.physics(t, stateVars)
        dx_dt = dstate_data[1]
        dpress_dt = dstate_data[2]
        dmass_burn_dt = dstate_data[3]
        dmass_exit_dt = dstate_data[4]

        return dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt

    def simulate(self):
        
        press0 = self.press_at*self.press_init
        dens0 = press0/(self.prop.r*self.prop.temp)
        vol_empty = self.case.volu - self.grains.get_volume()
        massb0 = dens0*vol_empty
        
        y0 = (0, self.press_at*self.press_init, massb0, 0)
        sol = RK45(motor.dydt, 0, y0, self.tspan, max_step=.1, rtol=1e-6)

        state_data, dstate_data, mass_flux_grain_end_data\
            = motor.physics(sol.t, sol.y)
        state_data, dstate_data, mass_flux_grain_end_data \
            = [state_data], [dstate_data], [mass_flux_grain_end_data]

        while sol.status == "running":
            sol.step()
            
            state_data_row, dstate_data_row, mass_flux_grain_end_data_row\
                = motor.physics(sol.t, sol.y)
            
            state_data.append(state_data_row)
            dstate_data.append(dstate_data_row)
            mass_flux_grain_end_data.append(mass_flux_grain_end_data_row)
            
            if state_data_row[2] <= 1.01*motor.press_at:  # if motor is done end
                break
        
        self.state_data = pd.DataFrame(state_data, columns=self.state_data_headers)
        self.dstate_data = pd.DataFrame(dstate_data, columns=self.dstate_data_headers)
        self.mass_flux_grain_end_data = pd.DataFrame(mass_flux_grain_end_data, columns=['t'] + ['grain{}'.format(i) for i in range(self.grains.num_grains)])
        self.data = pd.merge(self.state_data, self.dstate_data, on="t")
        self.calc_impulse()

    def calc_impulse(self):
        self.impulse = np.trapz(self.state_data['thrust'], x=self.state_data['t'])
    
    def motorcode(self):
        self.calc_impulse()
        ordal = ceil(log(self.impulse/2.5, 2))
        base_impulse = 2.5*2**(ordal-1)
        code = chr(ordal+65)
        per = "{:.0%} ".format(self.impulse/base_impulse-1)
        ave_thrust = self.impulse / max(self.state_data['t'])
        return per + code + '{:.0f}'.format(ave_thrust)

In [5]:
def inch2m(inch):
    return 0.0254 * inch
def psi2pa( psi ):
    return psi*6895

In [6]:
# full motor
# density: expected = 1688.5, achived = 1480.0, ratio = 0.877

# individual grain density kg.m3
g1 = 1511.905
g2 = 1572.36
g3 = 1552.9
g4 = 1542.15

# average density
density = ( g1 + g2 + g3 + g4 ) / 4
print('avg density = {:.1f}'.format(density))


case = Case(inch2m(3.239), inch2m(21.4))


# kickstart nominal density
expected_density = 1688.47

alpha = density/expected_density
phi = 1/alpha-1
print('alpha = {:.3f}, phi = {:.3f}'.format(alpha,phi))


grains = [ 
    Grain(case, inch2m(1.125), inch2m(5.010)), 
    Grain(case, inch2m(1.125), inch2m(5.139)), 
    Grain(case, inch2m(1.125), inch2m(5.073)), 
    Grain(case, inch2m(1.25), inch2m(5.081))
]

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)

# nozzle parameters
throat_diam = inch2m(.825)
exit_diam = inch2m(1.4)
expand = (exit_diam/throat_diam)**2
nozzle = Nozzle(throat_diam, expand, pi/12, .98)

motor = Motor(case, grains, kickstart, nozzle, mod=1.3)
motor.simulate()



# t, x, press, mass_burn, mass_exit, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
# vol_grains, vol_empty, dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
# mach, mass_flux, mach_exit, exit_velocity, thrust


data = motor.state_data.copy()

data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
data['thrust'] /= 4.448
print('max pressure = {:.1f} psi'.format(max(data['press'])))
print('max thrust = {:.1f} lbf'.format(max(data['thrust'])))

print( 'impulse = {:.0f} Nm'.format(motor.impulse))
print(motor.motorcode())

prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print( 'isp = {:.1f} s'.format(isp))
data.iplot(kind='line', x='t', y=['thrust', 'press'])

data.to_csv('theoredical_data.csv')

In [7]:
# full motor before
# density: expected = 1688.5, achived = 1480.0, ratio = 0.877

g1 = 1511.905
g2 = 1572.36
g3 = 1552.9
g4 = 1542.15


density = ( g1 + g2 + g3 + g4 ) / 4

length = inch2m(5)

case = Case(inch2m(3.239), inch2m(21.4))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3

expected_density = 1688.47

alpha = density/expected_density
phi = 1/alpha-1

print('alpha = {:.3f}, phi = {:.3f}'.format(alpha,phi))
grains = [Grain(case, inch2m(1.125), inch2m(5)) ]*3+[Grain(case, inch2m(1.25), inch2m(5))]

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=expected_density, k=1.21, r=351.25, temp=3500)

throat_diam = inch2m(.727081)

exit_diam = inch2m(1.4)
expand = (exit_diam/throat_diam)**2
nozzle = Nozzle(throat_diam, expand, pi/12, .98)

motor = Motor(case, grains, kickstart, nozzle, mod=1)
motor.simulate()



# t, x, press, mass_burn, mass_exit, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
# vol_grains, vol_empty, dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
# mach, mass_flux, mach_exit, exit_velocity, thrust


# motor.df.plot(x='t', y=['mach_exit', 'mach'])
# plt.grid(True)
# plt.show()

data = motor.state_data.copy()

data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
data['thrust'] /= 4.448
print('max pressure = {:.1f} psi'.format(max(data['press'])))
print('max thrust = {:.1f} lbf'.format(max(data['thrust'])))

print( 'impulse = {:.0f} Nm'.format(motor.impulse))
print(motor.motorcode())

prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print( 'isp = {:.1f} s'.format(isp))
data.iplot(kind='line', x='t', y=['thrust', 'press'])

In [8]:
# single grain Original 


length = inch2m(5)

case = Case(inch2m(3.239), inch2m(21.4))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3

# grain1 measurements
length = inch2m(4.99)
mass = .8357 # g
core = inch2m(1.25)


expected_density = 1688.47

grain = Grain(case, core, length)
density = mass/grain.volume
alpha = density/expected_density
phi = 1/alpha-1
print(density)
print('alpha = {:.3f}, phi = {:.3f}'.format(alpha,phi))


grains = [grain]

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)

throat_diam = inch2m(23/64)
exit_diam = inch2m(1.4)
expand = (exit_diam/throat_diam)**2


nozzle = Nozzle(throat_diam, expand, pi/12, 1)
print("expand = {:.3f}".format(expand))

motor = Motor(case, grains, kickstart, nozzle, mod=1)
motor.simulate()


data = motor.state_data.copy()
data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print('max pressure = {:.1f} psi'.format(max(data['press'])))
# print( 'impulse = {:.0f} Nm'.format(motor.impulse))
# print(motor.motorcode())
# print( 'isp = {:.1f} s'.format(isp))
t_burn = max(data['t'])
print("burn time = {:.3f}".format(t_burn))


data.iplot(kind='line', x='t', y=['thrust', 'press', 'thrust_press', 'thrust_flow', 'press_exit', 'mach_exit'])

data.to_csv('single_grain_theo_pre_mod.csv')




In [9]:
# single grain post analysis 


length = inch2m(5)

case = Case(inch2m(3.239), inch2m(21.4))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3

# grain1 measurements
length = inch2m(4.99)
mass = .8357 # g
core = inch2m(1.25)


expected_density = 1688.47

grain = Grain(case, core, length)
density = mass/grain.volume
alpha = density/expected_density
phi = 1/alpha-1

print('alpha = {:.3f}, phi = {:.3f}'.format(alpha,phi))


grains = [grain]

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)

throat_diam = inch2m(23/64)
exit_diam = inch2m(1.4)
expand = (exit_diam/throat_diam)**2


nozzle = Nozzle(throat_diam, expand, pi/12, 1)
print("expand = {:.3f}".format(expand))

motor = Motor(case, grains, kickstart, nozzle, mod=1.5)
motor.simulate()


data = motor.state_data.copy()
data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print('max pressure = {:.1f} psi'.format(max(data['press'])))
# print( 'impulse = {:.0f} Nm'.format(motor.impulse))
# print(motor.motorcode())
# print( 'isp = {:.1f} s'.format(isp))
t_burn = max(data['t'])
print("burn time = {:.3f}".format(t_burn))

data.iplot(kind='line', x='t', y=['thrust', 'press'])

data.to_csv('single_grain_theo_post_mod.csv')


In [10]:
# create .eng file

# Designation outter_diam length P mass_empty mass_full Manuf
# time thrust
# ;

# des = "Medusa_11.17"
# outter_diam = "98.0"
# length = str(inch2m(27))
# hardware_mass = 4.59827
# inital_mass = prop_mass + hardware_mass
# hardware_mass = '{:.4f}'.format(hardware_mass)
# inital_mass = '{:.4f}'.format(inital_mass)

# thrust_data = motor.state_data[['t', 'thrust']].clip(lower=0)


# with open(des+'.eng', 'w') as file:
#     top_line = ' '.join([des, outter_diam, length, 'P', hardware_mass, inital_mass, '\n'])
#     file.write(top_line)
#     thrust_data.to_csv(\
#                        path_or_buf=file, sep=' ', float_format='%.3f', header=False, index=False, \
#                        mode='a')

    
#     file.write( ';' )

In [11]:
# tiny motor

length = inch2m(2)

case = Case(inch2m(1), inch2m(4))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3


grain = Grain(case, inch2m(.375), length)
grains = [grain]

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)
# kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=expected_density, k=1.21, r=351.25, temp=3500)

nozzle = Nozzle(inch2m(3/16), 2, pi/12, .98)
# print( 'expand ratio optimized = {:.2f}'.format( nozzle.optimize_expand( kickstart, psi2pa(400) ) ) )

motor = Motor(case, grains, kickstart, nozzle)
motor.simulate()



# t, x, press, mass_burn, mass_exit, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
# vol_grains, vol_empty, dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
# mach, mass_flux, mach_exit, exit_velocity, thrust


# motor.df.plot(x='t', y=['mach_exit', 'mach'])
# plt.grid(True)
# plt.show()

data = motor.state_data.copy()
data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
data['thrust'] *= 102
print('max pressure = {:.1f} psi'.format(max(data['press'])))
print( 'impulse = {:.0f} Nm'.format(motor.impulse))
print(motor.motorcode())
prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81
print( 'isp = {:.1f} s'.format(isp))
data.iplot(kind='line', x='t', y=['thrust', 'press' ])

In [12]:
# 3 grain
# density: expected = 1688.5, achived = 1480.0, ratio = 0.877


density = 1480.0

length = inch2m(5)

case = Case(inch2m(3.239), inch2m(21.4))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3

grains = [ Grain(case, inch2m(1.125), inch2m(5)) ]*3

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)


throat_diam = inch2m(.825)
exit_diam = inch2m(1.4)
expand = (exit_diam/throat_diam)**2
nozzle = Nozzle(throat_diam, expand, pi/12, .98)

motor = Motor(case, grains, kickstart, nozzle, mod=1.5)
motor.simulate()



# t, x, press, mass_burn, mass_exit, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
# vol_grains, vol_empty, dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
# mach, mass_flux, mach_exit, exit_velocity, thrust



data = motor.state_data.copy()

data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
data['thrust'] /= 4.448

print('max pressure = {:.1f} psi'.format(max(data['press'])))
print('max thrust = {:.1f} lbf'.format(max(data['thrust'])))

print( 'impulse = {:.0f} Nm'.format(motor.impulse))
print(motor.motorcode())

prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print( 'isp = {:.1f} s'.format(isp))
data.iplot(kind='line', x='t', y=['thrust', 'press'])

In [13]:
# little 2 grain
# density: expected = 1688.5, achived = 1480.0, ratio = 0.877


density = 1480.0

length = inch2m(2)

case = Case(inch2m(1), inch2m(6))

# cherry limeade 1680.0 kg/m3
# kickstart 1688.47 kg/m3

grains = [ Grain(case, inch2m(3/8), inch2m(2)) ]*2

kickstart = Propellant(brc=1.8544e-05, bre=.36915, dens=density, k=1.21, r=351.25, temp=3500)


throat_diam = inch2m(3/16)
exit_diam = inch2m(3/8)
expand = (exit_diam/throat_diam)**2
nozzle = Nozzle(throat_diam, expand, pi/12, .98)

motor = Motor(case, grains, kickstart, nozzle, mod=1.5)
motor.simulate()



# t, x, press, mass_burn, mass_exit, dx_dt, dpress_dt, dmass_burn_dt, dmass_exit_dt,
# vol_grains, vol_empty, dvol_grains_dx, dvol_empty_dx, dvol_grains_dt, dvol_empty_dt,
# mach, mass_flux, mach_exit, exit_velocity, thrust



data = motor.state_data.copy()

data['press'] /= psi2pa(1)
data['press_exit'] /= psi2pa(1)
data['thrust'] /= 4.448

print('max pressure = {:.1f} psi'.format(max(data['press'])))
print('max thrust = {:.1f} lbf'.format(max(data['thrust'])))

print( 'impulse = {:.0f} Nm'.format(motor.impulse))
print(motor.motorcode())

prop_mass = motor.get_prop_mass()
isp = motor.impulse/prop_mass/9.81

print( 'isp = {:.1f} s'.format(isp))
data.iplot(kind='line', x='t', y=['thrust', 'press'])